In [1]:
import os
import datetime
# cambiamos las variables del sistema
spark = 'C:\\hlocal\\tdm\\spark\\hadoop\\spark-2.3.2-bin-hadoop2.7'
# en el path se añade
path = os.environ.get('PATH') 
path = path+ ';'+spark+'\\bin;'
os.environ['PATH'] = path
os.environ['SPARK_HOME']= spark 
os.environ['HADOOP_HOME']= spark 
os.environ['PYSPARK_DRIVER_PYTHON']= 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS']='notebook'

# si da problema con collect quizás haya que poner java_home
#os.environ['JAVA_HOME']= 'C:\\Program Files\\Java\\jdk1.8.0_151'
#labs = 'C:\\JDK\\jdk8-64bits'
#os.environ['JAVA_HOME']= labs
#os.environ['PATH'] = os.environ.get('JAVA_HOME')+'\\bin;'+path


import findspark
findspark.init()

import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
from pyspark.sql import Row
from pyspark.ml.regression import LinearRegressionModel
from pyspark.ml.feature import VectorAssembler
modelo = LinearRegressionModel.load("modelo")
print("Preparado!!")

Preparado!!


In [6]:
#calle="Paseo La Castellana"
#fecha="31/05/19 20:02"



def pred(calle,fecha):
    encabezados=[]
    encabezados2=[]


    fecha = datetime.datetime.strptime(fecha, '%d/%m/%y %H:%M')
    valores=[]
    horaactual=fecha.hour
    diaactual=fecha.weekday()
    dias=['L','M','X','J','V','S','D']
    for i in range(7):
        encabezados.append(dias[i])
        if(diaactual==i):
            valores.append(1)
        else:
            valores.append(0)
    for i in range(24):
        encabezados.append("hora_"+str(i))
        if(horaactual==i):
            valores.append(1)
        else:
            valores.append(0)
    encabezados.append("traficoactual")
    for i in range(len(encabezados)-1):
        encabezados2.append(encabezados[i])

    valores.append(0)
    valores2=[]
    valores2.append(valores)
    valoresrdd=sc.parallelize(valores2)
    valoresrdd.take(1)
    assembler = VectorAssembler(
            inputCols=encabezados2,
            outputCol='features')
    row_data = valoresrdd.map(lambda p: Row(
        hora_0=p[0],
        hora_1=p[1],
        hora_2=p[2],
        hora_3=p[3],
        hora_4=p[4],
        hora_5=p[5],
        hora_6=p[6],
        hora_7=p[7],
        hora_8=p[8],
        hora_9=p[9],
        hora_10=p[10],
        hora_11=p[11],
        hora_12=p[12],
        hora_13=p[13],
        hora_14=p[14],
        hora_15=p[15],
        hora_16=p[16],
        hora_17=p[17],
        hora_18=p[18],
        hora_19=p[19],
        hora_20=p[20],
        hora_21=p[21],
        hora_22=p[22],
        hora_23=p[23],
        L=p[24],
        M=p[25],
        X=p[26],
        J=p[27],
        V=p[28],
        S=p[29],
        D=p[30],
        traficoactual=p[31]
    ))

    row_data.take(1)
    valores_df = spark.createDataFrame(row_data)
    #valores_df.show()
    #valores_df.printSchema()
    #valores_df.describe().toPandas().transpose()
    datosprep =  assembler.transform(valores_df)
    datosprep = datosprep.select(['traficoactual','features'])
    prediccion=modelo.transform(datosprep)
    prediccionv=prediccion.select("prediction").take(1)[0].prediction
    etiquetastrafico=['Bajo', 'Medio','Alto']
    valorestrafico=[3, 10, 20]

    for i in range(len(valorestrafico)):
            if(valorestrafico[i]>prediccionv):
                etiqueta=etiquetastrafico[i]
                break
    return etiqueta

In [ ]:
from flask import Flask, request
from flask import Response
from flask import jsonify # <- `jsonify` instead of `json`
app = Flask(__name__)
puerto=50002
@app.route('/prediccion', methods = ['GET'])
def obtenerprediccion():
    calle=request.args.get('calle')
    fecha=request.args.get('fecha')
    return jsonify({"calle":calle,"fecha":fecha,"prediccion":pred(calle,fecha)}), 200
if __name__ == '__main__':
     app.run(port=puerto,host='0.0.0.0')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:50002/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Sep/2019 14:41:49] "GET /prediccion?calle=Paseo%20La%20Castellana&fecha=31/08/19%2020:00 HTTP/1.1" 200 -
